In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import os
tf.__version__
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/train.csv")

In [3]:
def clean_data(text):
    text = text.lower()
    text = re.sub(r"[-=+*\"#@!$%^&()`<>\[\]]","",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"there're","there are",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"how're","how are",text)
    text = re.sub(r"what're","what are",text)
    text = re.sub(r"where're","where ",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"that're","that are",text)
    text = re.sub(r"when're","when are",text)
    text = re.sub(r"how's","how is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"when's","when is",text)
    text = re.sub(r"won't","would not",text)
    text = re.sub(r"nt't","can not",text)
    text = re.sub(r"\'bout'","about",text)
    text = re.sub(r"\'till'","untill",text)
    text = re.sub(r"\'ll","will",text)
    text = re.sub(r"\'ve","have",text)
    text = re.sub(r"\'re","are",text)
    text = re.sub(r"\'d","would",text)
    text = re.sub(r"\."," ",text)
    text = re.sub(r"\,"," ",text)
    text = re.sub(r"\!"," ",text)
    text = re.sub(r"\?"," ",text)
    text = re.sub(r"\;"," ",text)
    text = re.sub(r"\:"," ",text)
    return text

    
df['text'] = df['text'].apply(clean_data)
df['augmented_text'] = df['augmented_text'].apply(clean_data)




In [5]:
df

,text,augmented_text
0,1 billion darling harbour redevelopment plans ...,1 billion darling hsarfbour redeevelopment p3...
1,1 billion desalination claim frivolous walsh,1 ibllion desalination lciam rfivloous walsh
2,1 billion wiped from nsw budget in accounting ...,1 billino wiped from nsw budget in accountngi...
3,1 million reward in hunt for bowraville killer,1 million reward in htun for boawrville killre
4,1 million wall to shield coal train noise,1 million wqli to shjele coal raib noise
...,...,...
768636,zygier arrested after leaking mossad work to a...,zygier arrectrd after leak9nh mossad eoek to a...
768637,zygier sabotaged mission to retrieve soldiers,zgyire sabotaged misisno to retrieve soldiers
768638,zylvester streaks field in alice cup,zylvester streaks field in ialce cup
768639,zynga adoption pushes bitcoin back over 1000,z6ngz adoption pushes fitcokn naxk over 1000


In [6]:
testing = df[72001:72006]
text_data = df.iloc[0:72000,]


In [7]:

g_text_tokenizer = Tokenizer()
g_text_tokenizer.fit_on_texts(text_data['augmented_text'])
g_text_word_index = g_text_tokenizer.word_index
train_sequences = g_text_tokenizer.texts_to_sequences(text_data['augmented_text'])
train_padded = pad_sequences(train_sequences, padding='post')

max_length_g = train_padded.shape[1]

text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(text_data['text'])
text_word_index = text_tokenizer.word_index
test_sequences = text_tokenizer.texts_to_sequences(text_data['text'])
test_padded = pad_sequences(test_sequences, padding='post')
max_length = test_padded.shape[1]




In [8]:
print(max_length_g,max_length)


14 14


In [10]:
g_vocab_size = len(g_text_word_index)+1
vocab_size = len(text_word_index)+1
print(g_vocab_size, vocab_size)

154250 31997


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=g_vocab_size, output_dim=2056, input_length=max_length_g),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1028, return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2056, activation='relu')),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1028, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size,activation='softmax'))
])
# learning_rate = 2
optimizer = tf.keras.optimizers.RMSprop()
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])

In [13]:
model.summary(expand_nested=True,
    show_trainable=True)

Model: "sequential"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 embedding (Embedding)       (None, 14, 2056)          31713800  Y          
                                                       0                    
                                                                            
 bidirectional (Bidirectiona  (None, 14, 2056)         25371040  Y          
 l)                                                                         
                                                                            
 time_distributed (TimeDistr  (None, 14, 2056)         4229192   Y          
 ibuted)                                                                    
                                                                            
 bidirectional_1 (Bidirectio  (None, 14, 2056)         25371040  Y          
 nal)                                                   

In [14]:
temp_test_padded = test_padded.reshape((test_padded.shape[0], test_padded.shape[1],1))
temp_train_padded = train_padded.reshape((train_padded.shape[0], train_padded.shape[1],1))


In [15]:
epoch = 20
model.fit(train_padded, test_padded, batch_size=128, epochs=epoch, validation_split=0.2)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:439: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 317138000 elements. This may consume a large amount of memory.
  num_elements)


450/450 [==============================] - 178s 378ms/step - loss: 3.5059 - accuracy: 0.5716 - val_loss: 3.2273 - val_accuracy: 0.6272
Epoch 2/20
450/450 [==============================] - 168s 374ms/step - loss: 2.3330 - accuracy: 0.7058 - val_loss: 2.5779 - val_accuracy: 0.7122
Epoch 3/20
450/450 [==============================] - 173s 384ms/step - loss: 1.7879 - accuracy: 0.7669 - val_loss: 2.3790 - val_accuracy: 0.7449
Epoch 4/20
450/450 [==============================] - 168s 374ms/step - loss: 1.4541 - accuracy: 0.8021 - val_loss: 2.2929 - val_accuracy: 0.7634
Epoch 5/20
450/450 [==============================] - 168s 374ms/step - loss: 1.2174 - accuracy: 0.8298 - val_loss: 2.3238 - val_accuracy: 0.7725
Epoch 6/20
450/450 [==============================] - 168s 374ms/step - loss: 1.0473 - accuracy: 0.8518 - val_loss: 2.2283 - val_accuracy: 0.7796
Epoch 7/20
450/450 [==============================] - 168s 374ms/step - loss: 0.9456 - accuracy: 0.8671 - val_loss: 2.1604 - val_accura

In [17]:
model.save('/content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_95_bidirectional_lstm_rnn')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_95_bidirectional_lstm_rnn/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_95_bidirectional_lstm_rnn/assets


In [18]:

test_sample = testing['augmented_text']

test_sample = g_text_tokenizer.texts_to_sequences(test_sample)
test_sample = pad_sequences(test_sample, maxlen=max_length_g, padding='post')


In [20]:
testing

,text,augmented_text
72001,beijing outbreak under control says official,beijing outbreak under ocntrlo assy official
72002,beijing petrol stations face olympic shutdown,biejnig petrol stations fcea olympic shutdown
72003,beijing plus 20 milestones for australian wome...,beijing plix 20 mil4qtones for australian wome...
72004,beijing pollution hits hazardous levels,beijgin pollution htsi hazardous levels
72005,beijing preparation goes awry for hoogie,ebiijng preparation goes awry for ohgoie


In [21]:
pre = model.predict(test_sample)
pre.shape

(5, 14, 31997)

In [23]:
def deTokenize(logits):
    index_to_words = {id: word for word, id in text_tokenizer.word_index.items()}
    index_to_words[0] = ''
    pre_index = np.argmax(logits, 1)
    return ' '.join(index_to_words[prediction] for prediction in pre_index)

In [32]:
print("Iutput:    ",testing['augmented_text'].iloc[0])
print("Actual:    ",testing['text'].iloc[0])
print("Predicted: ",deTokenize(pre[0]))

Iutput:     beijing outbreak under ocntrlo  assy official
Actual:     beijing outbreak under control  says official
Predicted:  beijing outbreak under control says official        
